In [ ]:
# Install Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-e233dqk6/unsloth_7272b4a2a8bd468fb58dff3445375c6a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-e233dqk6/unsloth_7272b4a2a8bd468fb58dff3445375c6a
  Resolved https://github.com/unslothai/unsloth.git to commit 1c0ad844f170f67c7cdf6f7a9465bafb0f9627df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-135M-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

model.print_trainable_parameters()

Unsloth 2025.11.2 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


trainable params: 39,075,840 || all params: 173,591,424 || trainable%: 22.5102


In [ ]:
from datasets import load_dataset

# Load Python coding dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")

# Check dataset structure
print(dataset[0])

{'instruction': 'Create a function to calculate the sum of a sequence of integers.', 'input': '[1, 2, 3, 4, 5]', 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum'}


In [ ]:
# Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # End of sequence token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Combine instruction, input, output using template
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply formatting to dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Add this line to disable wandb!
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


In [ ]:
# Start training
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 18,612 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,075,840 of 173,591,424 (22.51% trained)


Step,Training Loss
1,1.948900
2,1.823200
3,1.600100
4,1.593900
5,1.423800
6,1.484400
7,1.341200
8,1.200400
9,1.688200
10,1.147200


In [ ]:
# Enable fast inference mode
FastLanguageModel.for_inference(model)

# Test prompts
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write a Python function to calculate the factorial of a number.",  # instruction
        "",  # input
        "",  # output - leave blank for generation
    )
], return_tensors = "pt").to("cuda")

# Generate response
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
print(tokenizer.batch_decode(outputs))

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a Python function to calculate the factorial of a number.\n\n### Input:\n\n\n### Response:\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\n# Example usage:\nprint(factorial(5))  # Output: 120\n\n# Explanation:\n# 5! = 5 * 4 * 3 * 2 * 1 = 120\n\n# Explanation:\n# 5! = 5 * 4 * 3 * 2 * 1 = 120<|im_end|>']


In [ ]:
# Test multiple coding tasks
test_prompts = [
    "Write a Python function to reverse a string.",
    "Create a function to check if a number is prime.",
    "Write a function to find the largest element in a list."
]

for prompt in test_prompts:
    inputs = tokenizer(
        [alpaca_prompt.format(prompt, "", "")],
        return_tensors = "pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    print(f"\n{'='*50}")
    print(f"Instruction: {prompt}")
    print(f"{'='*50}")
    print(tokenizer.batch_decode(outputs)[0])


Instruction: Write a Python function to reverse a string.
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a Python function to reverse a string.

### Input:


### Response:
def reverse_string(string):
    reversed_string = ""
    for char in string:
        if char == ' ':
            reversed_string = reversed_string + char
        else:
            reversed_string = reversed_string + char
    return reversed_string<|im_end|>

Instruction: Create a function to check if a number is prime.
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Create a function to check if a number is prime.

### Input:


### Response:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            retu

In [ ]:
# Save LoRA adapters (small file ~100MB)
model.save_pretrained("smollm2_python_fullft")
tokenizer.save_pretrained("smollm2_python_fullft")


('smollm2_python_fullft/tokenizer_config.json',
 'smollm2_python_fullft/special_tokens_map.json',
 'smollm2_python_fullft/chat_template.jinja',
 'smollm2_python_fullft/vocab.json',
 'smollm2_python_fullft/merges.txt',
 'smollm2_python_fullft/added_tokens.json',
 'smollm2_python_fullft/tokenizer.json')

In [ ]:
# Load the fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "smollm2_python_fullft",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

# Test again
inputs = tokenizer(
    [alpaca_prompt.format("Write a function to calculate fibonacci numbers.", "", "")],
    return_tensors = "pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
print(tokenizer.batch_decode(outputs))

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a function to calculate fibonacci numbers.\n\n### Input:\n\n\n### Response:\ndef fibonacci(n):\n    if n < 0:\n        return 0\n    elif n == 1:\n        return 1\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n\n# Example usage:\nprint(fibonacci(10))<|im_end|>']
